# Aprendizaje Supervisado

Proyecto de gestión de inventarios usando SQL Server

## 1 Conexión

La conexión mostrada en este proyecto Actualmente se encuentra implementada para un servicio local, por lo tanto puede ser objeto de cambios en cuanto se despliegue la base de datos en un servidor EC2

In [1]:
import pyodbc
import pandas as pd
import numpy as np

driver = 'SQL Server'
server = 'MARCO'
db1 = 'AI_Eafit'
tcon = 'yes'
uname = 'aieafit'
pword = 'marANT1502'

cnxn = pyodbc.connect(driver='{SQL Server}', host=server, database=db1,
                      trusted_connection=tcon, user=uname, password=pword)

# 2 Modelos

## 2.1 Regresión Logística

Una vez se ejecuta la conexión se deben seleccionar los datos establecidos en las variables de respuesta escogidas, este paso se dividirá por Queries de cada selección y se utilizará la biblioteca Microsoftml para implementar los modelos

### dbo.Fact_compra

In [2]:
query_compras = "SELECT TOP 1000000 Id_Producto, Id_Bodega, Id_Comprador, Fecha_Orden_Compra, Compra_Cantidad, Compra_Pesos, Precio_Unitario_Moneda, Linea_Completa, Linea_Perfecta, Lead_Time_Teorico_Compra, Lead_Time_Real_Max_Compra FROM dbo.T_Fact_compra"
df_compras = pd.read_sql(query_compras, cnxn)

df_compras['Fecha_Orden_Compra'] = df_compras['Fecha_Orden_Compra'].str.replace(r'-', '')
df_compras['Fecha_Orden_Compra'] = df_compras['Fecha_Orden_Compra'].astype('int64')

df_compras.head()

,Id_Producto,Id_Bodega,Id_Comprador,Fecha_Orden_Compra,Compra_Cantidad,Compra_Pesos,Precio_Unitario_Moneda,Linea_Completa,Linea_Perfecta,Lead_Time_Teorico_Compra,Lead_Time_Real_Max_Compra
0,2353,3,15348,20181106,120.0,152640.0,1272.0,0.0,0.0,0.0,0.0
1,4769,3,15348,20181106,6.0,61566.0,10261.0,0.0,0.0,0.0,0.0
2,8561,3,15348,20181106,24.0,71544.0,2981.0,0.0,0.0,0.0,0.0
3,6145,4,5964,20181109,12.0,112320.0,9360.0,1.0,1.0,7.0,7.0
4,2353,4,5964,20181109,216.0,274752.0,1272.0,1.0,1.0,7.0,7.0


#### Separación de los datos

se divide el dataframe en un set para entrenamiento y uno de pruebas

In [3]:
from sklearn.model_selection import train_test_split
bc_train_compras, bc_test_compras = train_test_split(df_compras, test_size=0.2)
print("# of rows in training set = ",bc_train_compras.size)
print("# of rows in test set = ",bc_test_compras.size)

# of rows in training set =  1294964
# of rows in test set =  323741


### Entrenamiento de modelo

Se entrena el modelo usando el paquete Microsoft for ML

In [6]:
from microsoftml import rx_fast_trees

features_compras = df_compras.columns.drop(["Linea_Perfecta"])
model_compras = rx_fast_trees("Linea_Perfecta ~ " + "+".join(features_compras), data=bc_train_compras, )

Beginning processing data.
Rows Read: 117724, Read Time: 0.001, Transform Time: 0
Beginning processing data.
Not adding a normalizer.
Making per-feature arrays
Changing data from row-wise to column-wise
Beginning processing data.
Rows Read: 117724, Read Time: 0, Transform Time: 0
Beginning processing data.
Processed 117724 instances
Binning and forming Feature objects
Reserved memory for tree learner: 232284 bytes
Starting to train ...
Not training a calibrator because it is not needed.
Elapsed time: 00:00:01.2099236
Elapsed time: 00:00:00.0253385


### Predecir

se predice el modelo y se imprimen las probabilidades encontradas en la regresión implementada (no binaria)

In [7]:
from microsoftml import rx_predict

prediction_compras = rx_predict(model_compras, data=bc_test_compras)

prediction_compras.head()

Beginning processing data.
Rows Read: 29431, Read Time: 0, Transform Time: 0
Beginning processing data.
Elapsed time: 00:00:00.2965266
Finished writing 29431 rows.
Writing completed.


,PredictedLabel,Score.1,Probability.1
0,0,-9.915150,0.018596
1,0,-16.141308,0.001568
2,0,-16.080351,0.001606
3,0,-3.693698,0.185808
4,0,-10.094634,0.017330


### dbo.Fact_Ventas

In [8]:
query_ventas = "SELECT TOP 1000000 Cod_Compañia, Id_Producto, Id_Vendedor, Id_Bodega, Fecha_Factura, Venta_Cantidad, Venta_Neta, Venta_Costo_Pesos, Venta_margen_pesos, Pedido_Numero_Linea FROM dbo.T_Fact_venta"
df_ventas = pd.read_sql(query_ventas, cnxn)

df_ventas['Fecha_Factura'] = df_ventas['Fecha_Factura'].str.replace(r'-', '')
df_ventas['Fecha_Factura'] = df_ventas['Fecha_Factura'].astype('int64')

df_ventas.head()

,Cod_Compañia,Id_Producto,Id_Vendedor,Id_Bodega,Fecha_Factura,Venta_Cantidad,Venta_Neta,Venta_Costo_Pesos,Venta_margen_pesos,Pedido_Numero_Linea
0,1,3285,18275,12,20181218,6.0,60323.0,52183.32,8139.68,933329.0
1,1,2196,18275,12,20181218,6.0,70560.0,56448.00,14112.00,933303.0
2,1,2199,18275,12,20181218,12.0,141120.0,112896.00,28224.00,933306.0
3,1,2263,18275,12,20181218,2.0,60480.0,48384.00,12096.00,933310.0
4,1,7275,18275,12,20181218,25.0,106245.0,82724.00,23521.00,933325.0


#### Separación de los datos

se divide el dataframe en un set para entrenamiento y uno de pruebas

In [9]:
from sklearn.model_selection import train_test_split
bc_train_ventas, bc_test_ventas = train_test_split(df_ventas, test_size=0.2)
print("# of rows in training set = ",bc_train_ventas.size)
print("# of rows in test set = ",bc_test_ventas.size)

# of rows in training set =  8000000
# of rows in test set =  2000000


### Entrenamiento de modelo

Se entrena el modelo usando el paquete Microsoft for ML

In [12]:
from microsoftml import rx_fast_trees

features_ventas = df_ventas.columns.drop(["Id_Producto"])
model_ventas = rx_fast_trees("Id_Producto ~ " + "+".join(features_ventas), data=bc_train_ventas, method="regression")

'unbalanced_sets' ignored for method 'regression'
Not adding a normalizer.
Making per-feature arrays
Changing data from row-wise to column-wise
Beginning processing data.
Rows Read: 800000, Read Time: 0.001, Transform Time: 0
Beginning processing data.
Processed 724638 instances
Binning and forming Feature objects
Reserved memory for tree learner: 260676 bytes
Starting to train ...
Not training a calibrator because it is not needed.
Elapsed time: 00:00:05.8225426
Elapsed time: 00:00:00.0300966


### Predecir

se predice el modelo y se imprimen las probabilidades encontradas en la regresión implementada (no binaria)

In [13]:
from microsoftml import rx_predict

prediction_compras = rx_predict(model_compras, data=bc_test_compras)

prediction_compras.head()

Beginning processing data.
Rows Read: 29431, Read Time: 0, Transform Time: 0
Beginning processing data.
Elapsed time: 00:00:00.2033937
Finished writing 29431 rows.
Writing completed.


,PredictedLabel,Score.1,Probability.1
0,0,-9.915150,0.018596
1,0,-16.141308,0.001568
2,0,-16.080351,0.001606
3,0,-3.693698,0.185808
4,0,-10.094634,0.017330


### dbo.Fact_pedido

In [14]:
query_pedido = "SELECT TOP 1000000 Id_Producto, Id_Vendedor, Id_Bodega, Fecha_Entrega_Planeada, Pedido_Descuento_Pesos, Pedido_Cantidad_Pendiente, conteo_linea_pedido, Linea_Completa, Linea_Perfecta, Pedido_Cantidad_Facturada, Pedido_Pesos_Facturados, Lead_time_Pedido_teorico, Lead_time_pedido_real_max FROM dbo.T_Fact_pedido"
df_pedido = pd.read_sql(query_pedido, cnxn)

df_pedido['Fecha_Entrega_Planeada'] = df_pedido['Fecha_Entrega_Planeada'].str.replace(r'-', '')
df_pedido['Fecha_Entrega_Planeada'] = df_pedido['Fecha_Entrega_Planeada'].astype('int64')

df_pedido.head()

,Id_Producto,Id_Vendedor,Id_Bodega,Fecha_Entrega_Planeada,Pedido_Descuento_Pesos,Pedido_Cantidad_Pendiente,conteo_linea_pedido,Linea_Completa,Linea_Perfecta,Pedido_Cantidad_Facturada,Pedido_Pesos_Facturados,Lead_time_Pedido_teorico,Lead_time_pedido_real_max
0,3108,5958,4,20151121,0.0,0.0,1.0,1.0,1.0,12.0,313920.0,15.0,1.0
1,2504,5955,3,20151216,0.0,0.0,1.0,1.0,1.0,1.0,51765.0,1.0,0.0
2,2204,5958,4,20151124,0.0,0.0,1.0,1.0,1.0,24.0,210480.0,1.0,0.0
3,4983,5958,4,20151126,0.0,0.0,1.0,1.0,1.0,1.0,102181.0,1.0,0.0
4,2997,5958,4,20140806,0.0,0.0,1.0,1.0,1.0,2000.0,230000.0,1.0,1.0


#### Separación de los datos

se divide el dataframe en un set para entrenamiento y uno de pruebas

In [15]:
from sklearn.model_selection import train_test_split
bc_train_pedido, bc_test_pedido = train_test_split(df_pedido, test_size=0.2)
print("# of rows in training set = ",bc_train_pedido.size)
print("# of rows in test set = ",bc_test_pedido.size)

# of rows in training set =  10400000
# of rows in test set =  2600000


### Entrenamiento de modelo

Se entrena el modelo usando el paquete Microsoft for ML

In [16]:
from microsoftml import rx_fast_trees

features_pedido = df_pedido.columns.drop(["Linea_Perfecta"])
model_pedido = rx_fast_trees("Linea_Perfecta ~ " + "+".join(features_pedido), data=bc_train_pedido)

Beginning processing data.
Rows Read: 800000, Read Time: 0, Transform Time: 0
Beginning processing data.
Not adding a normalizer.
Making per-feature arrays
Changing data from row-wise to column-wise
Beginning processing data.
Rows Read: 800000, Read Time: 0, Transform Time: 0
Beginning processing data.
Processed 800000 instances
Binning and forming Feature objects
Reserved memory for tree learner: 320112 bytes
Starting to train ...
Not training a calibrator because it is not needed.
Elapsed time: 00:00:07.7115646
Elapsed time: 00:00:00.0272237


### Predecir

se predice el modelo y se imprimen las probabilidades encontradas en la regresión implementada (no binaria)

In [17]:
from microsoftml import rx_predict

prediction_pedido = rx_predict(model_pedido, data=bc_test_pedido)

prediction_pedido.head()

Beginning processing data.
Rows Read: 200000, Read Time: 0, Transform Time: 0
Beginning processing data.
Elapsed time: 00:00:00.5694724
Finished writing 200000 rows.
Writing completed.


,PredictedLabel,Score.1,Probability.1
0,0,-31.374744,0.000004
1,1,22.729147,0.999887
2,1,21.532785,0.999818
3,0,-31.374744,0.000004
4,1,23.228197,0.999908


### dbo.Fact_Inventario

In [18]:
query_inventario = "SELECT TOP 1000000 Fecha_Inventario, Id_Bodega, Id_Producto, Inventario_Cantidad, Inventario_Pesos FROM dbo.T_Fact_Inventario"
df_inventario = pd.read_sql(query_inventario, cnxn)

df_inventario['Fecha_Inventario'] = df_inventario['Fecha_Inventario'].str.replace(r'-', '')
df_inventario['Fecha_Inventario'] = df_inventario['Fecha_Inventario'].astype('int64')

df_inventario.head()

,Fecha_Inventario,Id_Bodega,Id_Producto,Inventario_Cantidad,Inventario_Pesos
0,20170131,1,4621,-8.0,-138603.52
1,20170131,1,3297,-1.0,-81704.25
2,20170131,1,3544,9.0,82800.00
3,20170131,5,2931,-13.0,-104000.00
4,20170131,3,4810,-24.0,-151200.72


#### Separación de los datos

se divide el dataframe en un set para entrenamiento y uno de pruebas

In [19]:
from sklearn.model_selection import train_test_split
bc_train_inventario, bc_test_inventario = train_test_split(df_inventario, test_size=0.2)
print("# of rows in training set = ",bc_train_inventario.size)
print("# of rows in test set = ",bc_test_inventario.size)

# of rows in training set =  4000000
# of rows in test set =  1000000


### Entrenamiento de modelo

Se entrena el modelo usando el paquete Microsoft for ML

In [21]:
from microsoftml import rx_fast_trees

features_inventario = df_inventario.columns.drop(["Inventario_Cantidad"])
model_inventario = rx_fast_trees("Inventario_Cantidad ~ " + "+".join(features_inventario), data=bc_train_inventario, method="regression")

'unbalanced_sets' ignored for method 'regression'
Not adding a normalizer.
Making per-feature arrays
Changing data from row-wise to column-wise
Beginning processing data.
Rows Read: 800000, Read Time: 0, Transform Time: 0
Beginning processing data.
Processed 800000 instances
Binning and forming Feature objects
Reserved memory for tree learner: 87984 bytes
Starting to train ...
Not training a calibrator because it is not needed.
Elapsed time: 00:00:03.4242193
Elapsed time: 00:00:00.0275564


### Predecir

se predice el modelo y se imprimen las probabilidades encontradas en la regresión implementada (no binaria)

In [22]:
from microsoftml import rx_predict

prediction_compras = rx_predict(model_compras, data=bc_test_compras)

prediction_compras.head()

Beginning processing data.
Rows Read: 29431, Read Time: 0, Transform Time: 0
Beginning processing data.
Elapsed time: 00:00:00.2303205
Finished writing 29431 rows.
Writing completed.


,PredictedLabel,Score.1,Probability.1
0,0,-9.915150,0.018596
1,0,-16.141308,0.001568
2,0,-16.080351,0.001606
3,0,-3.693698,0.185808
4,0,-10.094634,0.017330
